<a href="https://colab.research.google.com/github/A00785001/TC5035/blob/main/004-Loop_Closure_Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loop Closure Dataset Generation: Time Alignment, Pairing & Labeling
## Phase 1.5: Supervised Learning Dataset Preparation for Jetson Nano Training Pipeline

**Pipeline Phase:** Feature Extraction → **[THIS NOTEBOOK]** → Fusion MLP Training → Deployment  
**Target Hardware:** Waveshare Jetbot AI Pro Kit (Jetson Nano)  
**SLAM System:** Google Cartographer (2D)  
**Training Platform:** Vertex AI

---

## 📋 NOTEBOOK DOCUMENTATION

### Purpose

This notebook transforms independently extracted multi-modal features (camera + LiDAR) into a supervised learning dataset for training a loop closure detection classifier. It bridges the gap between feature extraction and neural network training by performing temporal alignment, intelligent pairing, and ground truth labeling.

---

### Required Inputs

#### 1. Extracted Features (HDF5 Format)

**File:** `features.h5` (generated from feature extraction pipeline)

**Structure:**
```
features.h5
├── camera/
│   ├── features [N_cam, 1280]    # MobileNetV2 embeddings (L2 normalized)
│   ├── timestamps [N_cam]        # ROS timestamps (float64, seconds)
│   └── filenames [N_cam]         # Source image filenames (strings)
└── lidar/
    ├── features [N_lid, 256]     # 1D CNN descriptors (L2 normalized)
    ├── timestamps [N_lid]        # ROS timestamps (float64, seconds)
    └── filenames [N_lid]         # Source scan filenames (strings)
```

**Properties:**
- All features are L2 normalized (||f|| = 1.0)
- Timestamps are in ROS time format (seconds since epoch)
- Features were extracted independently without temporal alignment
- Typical size: 100-300 frames per modality per session

---

#### 2. ROS Bag File (ROS1 Format)

**File:** `session_data.bag` (from data collection session)

**Critical Topics Required:**

| Topic | Type | Rate | Purpose |
|-------|------|------|----------|
| `/trajectory_node_list` | visualization_msgs/MarkerArray | ~0.9 Hz | SLAM trajectory nodes with poses |
| `/constraint_list` | visualization_msgs/MarkerArray | ~0.36 Hz | SLAM constraints (loop closures) |
| `/scan` | sensor_msgs/LaserScan | ~0.7 Hz | LiDAR validation (optional) |
| `/csi_cam_0/image_raw/compressed` | sensor_msgs/CompressedImage | ~0.22 Hz | Camera validation (optional) |

**Ground Truth Source:**  
Cartographer SLAM publishes loop closure detections as `INTER_SUBMAP` constraints in `/constraint_list`. These represent confirmed spatial correspondences that the robot has revisited a previous location and serve as ground truth positive labels.

---

#### 3. Session Metadata

**Required Information:**
- Session ID (e.g., `20251016_133216`)
- Session duration and environment (for validation)
- Map dimensions (for spatial thresholds)

---

### What This Notebook Does

#### **Phase 1: Time Alignment & Synchronization**

**Challenge:** Multi-rate asynchronous sensors produce features and trajectory nodes at different rates:
- Camera: ~0.22 Hz (every 4.5 seconds)
- LiDAR: ~0.7 Hz (every 1.4 seconds)  
- Trajectory nodes: ~0.9 Hz (every 1.1 seconds)

**Solution:** Bidirectional nearest neighbor matching with temporal tolerance

**Steps:**
1. Parse ROS bag to extract all trajectory nodes (node_id, timestamp, pose)
2. Load camera and LiDAR features with their timestamps
3. Align features to nodes using KD-tree nearest neighbor search (max offset: 0.5s)
4. Create unified database mapping node_id → {camera_feature, lidar_feature, pose}
5. Filter to retain only nodes with both modalities present

**Output:** Temporally aligned multi-modal feature database

---

#### **Phase 2: Intelligent Pairing Strategy**

**Challenge:** Create balanced training pairs that help the model learn robust loop closure detection

**Solution:** Three-tier pairing strategy based on spatial and perceptual characteristics

**Pair Types:**

1. **Positive Pairs (Target: 30%)**
   - Source: Cartographer INTER_SUBMAP constraints
   - Criteria: Spatial distance < 2.0m, angular distance < π/2
   - Label: 1 (loop closure)
   - Purpose: Learn what true loop closures look like

2. **Easy Negative Pairs (Target: 35%)**
   - Source: Random sampling from trajectory
   - Criteria: Spatial distance > 5.0m, temporal distance > 5.0s
   - Label: 0 (not loop closure)
   - Purpose: Learn basic spatial discrimination

3. **Hard Negative Pairs (Target: 35%)**
   - **Type A - Perceptual Aliasing:** High feature similarity (cosine > 0.7) but spatially distant (> 3.0m)
   - **Type B - Different Viewpoints:** Same location (< 2.0m) but different orientation (> π/2) and temporally distinct (> 5.0s)
   - Label: 0 (not loop closure)
   - Purpose: Learn robustness against confusing situations (similar-looking places, revisits from different angles)

**Output:** Balanced set of (query_node_id, candidate_node_id, label) triplets

---

#### **Phase 3: Pairwise Feature Computation & Labeling**

**Challenge:** Convert pair information into trainable feature vectors

**Solution:** Absolute difference encoding with temporal splitting

**Steps:**
1. For each pair (query, candidate):
   - Retrieve concatenated features: f_query = [1280D_cam + 256D_lidar] = 1536D
   - Retrieve concatenated features: f_candidate = 1536D
   - Compute pairwise feature: abs(f_query - f_candidate) = 1536D
2. Create train/validation/test splits using temporal ordering:
   - Train: First 60% of trajectory
   - Validation: Middle 20% of trajectory  
   - Test: Last 20% of trajectory
3. Package into final dataset with metadata

**Why absolute difference?**
- Symmetric: |f_a - f_b| = |f_b - f_a| (order doesn't matter)
- Range: [0, √2] for L2-normalized features
- Small values → similar features → likely loop closure
- Large values → different features → likely not loop closure

**Output:** Final training-ready dataset

---

### Expected Outputs

#### Primary Output

**File:** `loop_closure_dataset.pkl` (Python pickle format)

**Structure:**
```python
{
  'metadata': {
    'session_id': str,
    'creation_date': str,
    'num_valid_nodes': int,
    'num_total_pairs': int,
    'positive_ratio': float,
    'feature_dim': int  # 1536
  },
  'train': {
    'features': np.array [N_train, 1536],  # Pairwise features
    'labels': np.array [N_train],          # Binary labels
    'pair_info': list of dicts             # (query_id, candidate_id, distance, etc.)
  },
  'val': { ... },      # Same structure as train
  'test': { ... },     # Same structure as train
  'node_database': dict  # Full node info for analysis
}
```

---

#### Diagnostic Outputs

**Files Generated:**
- `alignment_report.txt` - Phase 1 statistics (alignment rates, time offsets)
- `pairing_report.txt` - Phase 2 statistics (pair counts, spatial distributions)
- `validation_report.txt` - Quality assurance checks
- `dataset_diagnostics.png` - Visualization plots:
  - Spatial distribution of pairs on map
  - Feature similarity distributions
  - Label balance per split
  - Temporal coverage analysis

---

### Success Criteria

The dataset is ready for Phase 2 (MLP Training) when:

✅ **Alignment Rate:** >80% of features successfully aligned to trajectory nodes  
✅ **Valid Nodes:** >50 nodes with both camera and LiDAR features  
✅ **Positive Pairs:** At least 20 INTER_SUBMAP constraints found  
✅ **Label Balance:** 25-35% positive pairs (target: 30%)  
✅ **Hard Negatives:** At least 30% of total pairs are hard negatives  
✅ **No Data Leakage:** Train/val/test splits are temporally ordered  
✅ **Feature Quality:** Mean pairwise feature in range [0.2, 0.5], std > 0.1

---

### Pipeline Context

```
┌─────────────────────────────────────────────────────────────┐
│  Phase 0: Data Collection                                   │
│  • Robot navigation with SLAM                               │
│  • Output: session_data.bag                                 │
└────────────────────┬────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────┐
│  Phase 1: Feature Extraction                                │
│  • MobileNetV2 (camera) + 1D CNN (LiDAR)                    │
│  • Output: features.h5                                      │
└────────────────────┬────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────┐
│  Phase 1.5: Dataset Generation [THIS NOTEBOOK]              │
│  • Time alignment                                           │
│  • Intelligent pairing                                      │
│  • Ground truth labeling                                    │
│  • Output: loop_closure_dataset.pkl                         │
└────────────────────┬────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────┐
│  Phase 2: Fusion MLP Training                               │
│  • Architecture: 1536→512→128→1                             │
│  • Loss: BCE + Hard Negative Mining                         │
│  • Output: fusion_mlp.pth                                   │
└────────────────────┬────────────────────────────────────────┘
                     ↓
┌─────────────────────────────────────────────────────────────┐
│  Phase 3: Export & Deployment                               │
│  • Convert to ONNX → TensorRT FP16                          │
│  • Deploy to Jetson Nano                                    │
└─────────────────────────────────────────────────────────────┘
```

---

### Key Parameters

**Time Alignment:**
- `max_time_offset`: 0.5 seconds (allows one frame lag for asynchronous sensors)

**Spatial Thresholds:**
- Positive pairs: distance < 2.0m, angle < 90°
- Easy negatives: distance > 5.0m
- Hard negatives (Type A): distance > 3.0m, cosine similarity > 0.7
- Hard negatives (Type B): distance < 2.0m, angle > 90°

**Dataset Composition:**
- Positive: 30% (1:2 ratio per pipeline design)
- Easy negative: 35%
- Hard negative: 35%

**Train/Val/Test Split:**
- 60% / 20% / 20% based on temporal trajectory progression

---

### Notes & Considerations

⚠️ **Cartographer Message Format:** The `/trajectory_node_list` and `/constraint_list` topics use `visualization_msgs/MarkerArray` format (for RViz visualization) rather than native SLAM messages. This notebook includes specialized parsing logic to extract structured data from marker namespaces and IDs.

⚠️ **Feature Normalization:** All input features must be L2 normalized. The pairwise absolute difference will have range [0, √2].

⚠️ **Temporal Ordering:** Train/val/test splits respect temporal ordering to prevent data leakage. The model should generalize to future trajectories, not memorize past ones.

⚠️ **Hard Negative Importance:** Hard negatives are critical for robustness. Without them, the model may fail on perceptually similar places or revisits from different angles.

⚠️ **Session Requirements:** For robust training, combine multiple sessions (5-10) to reach 5,000-10,000 total pairs. Single session provides proof-of-concept but may overfit.

---

## <font color='#2E86AB'>▼ 1. Initialization and Setup</font>

In [ ]:
# Install required packages
!pip install --quiet rosbags opencv-python pillow numpy matplotlib tqdm seaborn pandas h5py scipy scikit-learn

In [ ]:
# Import standard libraries
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import pickle
import h5py
from datetime import datetime
from tqdm import tqdm
from collections import defaultdict
import io
import os
from pathlib import Path

# Import scientific computing
from scipy.spatial import KDTree
from scipy.spatial.distance import cosine, euclidean
from sklearn.model_selection import train_test_split

# Import ROS bag handling
from rosbags.rosbag1 import Reader
from rosbags.typesys import Stores, get_typestore

print("✅ All libraries loaded successfully!")

In [ ]:
# Initialize typestore for ROS1 message deserialization
typestore = get_typestore(Stores.ROS1_NOETIC)
print("✅ Typestore initialized for ROS1 Noetic")

## <font color='#2E86AB'>▼ 2. Storage Mounting</font>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## <font color='#2E86AB'>▼ 3. ROS Bag and Feature File Loading</font>

In [ ]:
# Specify the path to the sessions folder
data_path = "/content/drive/MyDrive/DATA/Artificial_Intelligence/MNA-V/Subjects/TC5035-Proyecto_Integrador/TC5035.data/jetbot/"

In [ ]:
# Specify the session
session_id = "20251016_133216"
print(f"Using session: {session_id}")

In [ ]:
# Construct paths
session_path = Path(data_path) / session_id
bag_path = session_path / "session_data.bag"
features_path = session_path / "features.h5"  # Assumes features.h5 is in same directory

# Verify files exist
if not bag_path.exists():
    raise FileNotFoundError(f"ROS bag not found: {bag_path}")
if not features_path.exists():
    raise FileNotFoundError(f"Features file not found: {features_path}")

print(f"✅ ROS Bag: {bag_path}")
print(f"✅ Features: {features_path}")

In [ ]:
# Get basic bag information
with Reader(bag_path) as reader:
    bag_name = bag_path.name
    file_size_mb = bag_path.stat().st_size / (1024 * 1024)
    duration_sec = (reader.end_time - reader.start_time) * 1e-9
    total_messages = reader.message_count
    unique_topics = {conn.topic for conn in reader.connections}
    avg_message_rate = total_messages / duration_sec if duration_sec > 0 else 0

print("\n" + "="*60)
print("ROS BAG INFORMATION")
print("="*60)
print(f"File: {bag_name}")
print(f"Size: {file_size_mb:.2f} MB")
print(f"Duration: {duration_sec:.2f} seconds ({duration_sec/60:.2f} minutes)")
print(f"Total Messages: {total_messages:,}")
print(f"Unique Topics: {len(unique_topics)}")
print(f"Average Message Rate: {avg_message_rate:.2f} msg/sec")
print("="*60)

## <font color='#2E86AB'>▼ 4. ROS Bag Basic EDA</font>

Quick exploratory data analysis to understand the bag contents before proceeding with dataset generation.

### 4.1 Topic Overview

In [ ]:
# Extract topic information
print("\n" + "="*60)
print("TOPIC ANALYSIS")
print("="*60)

with Reader(bag_path) as reader:
    topic_data = []
    for idx, conn in enumerate(reader.connections):
        topic_data.append({
            'Connection_Index': idx,
            'Topic': conn.topic,
            'Message Type': conn.msgtype,
            'Message Count': conn.msgcount,
            'Frequency (Hz)': conn.msgcount / duration_sec if duration_sec > 0 else 0
        })

    df_topics = pd.DataFrame(topic_data)
    df_topics = df_topics.sort_values('Message Count', ascending=False)

    print("\n📊 Topics in ROS Bag:")
    print(df_topics.to_string(index=False))

    # Save to CSV
    df_topics.to_csv('topic_overview.csv', index=False)
    print("\n✅ Topic overview saved to: topic_overview.csv")

### 4.2 Message Rate Analysis

In [ ]:
# Analyze message timing and rates
print("\n" + "="*60)
print("MESSAGE RATE ANALYSIS")
print("="*60)

with Reader(bag_path) as reader:
    # Collect timestamps per connection
    connection_timestamps = defaultdict(list)

    print("\nCollecting message timestamps...")
    for connection, timestamp, rawdata in tqdm(reader.messages(), total=reader.message_count):
        # Find connection index
        conn_idx = None
        for idx, conn in enumerate(reader.connections):
            if (conn.topic == connection.topic and
                conn.msgtype == connection.msgtype and
                conn.msgcount == connection.msgcount):
                conn_idx = idx
                break

        if conn_idx is not None:
            connection_timestamps[conn_idx].append(timestamp * 1e-9)  # Convert to seconds

    # Calculate statistics
    print(f"\n{'Topic':<40} {'Description':<30} {'Mean Δt (s)':<12} {'Std Δt (s)':<12}")
    print("-" * 94)

    for conn_idx, timestamps in sorted(connection_timestamps.items()):
        if len(timestamps) > 1:
            intervals = np.diff(timestamps)

            # Get topic info
            topic_row = df_topics[df_topics['Connection_Index'] == conn_idx].iloc[0]
            topic_name = topic_row['Topic']

            print(f"{topic_name:<40} {np.mean(intervals):<12.4f} {np.std(intervals):<12.4f}")

## <font color='#2E86AB'>▼ 5. Cartographer Data Validation</font>

**Purpose:** Validate that Cartographer SLAM data is consistent, complete, and suitable for generating ground truth labels.

**What we validate:**
1. `/trajectory_node_list` topic exists and contains valid nodes
2. `/constraint_list` topic exists and contains INTER_SUBMAP constraints (loop closures)
3. Node IDs are consistent and can be matched between topics
4. Spatial data (poses) is reasonable and covers the map
5. Temporal data is consistent with sensor timestamps
6. Sufficient INTER_SUBMAP constraints exist for positive pairs

**Critical for Success:** Without valid Cartographer data, we cannot generate ground truth labels for loop closure detection.

### 5.1 Check Required Topics Existence

In [ ]:
print("="*60)
print("CARTOGRAPHER DATA VALIDATION - PHASE 1: TOPIC CHECK")
print("="*60)

required_topics = {
    '/trajectory_node_list': 'visualization_msgs/MarkerArray',
    '/constraint_list': 'visualization_msgs/MarkerArray'
}

found_topics = {}
validation_passed = True

with Reader(bag_path) as reader:
    for topic_name, expected_type in required_topics.items():
        found = False
        for conn in reader.connections:
            if conn.topic == topic_name:
                found = True
                found_topics[topic_name] = {
                    'type': conn.msgtype,
                    'count': conn.msgcount,
                    'matches_expected': conn.msgtype == expected_type
                }
                break

        if not found:
            print(f"❌ FAILED: Topic '{topic_name}' not found in bag file")
            validation_passed = False
        else:
            info = found_topics[topic_name]
            if info['matches_expected']:
                print(f"✅ PASS: Topic '{topic_name}' found")
                print(f"   - Type: {info['type']}")
                print(f"   - Messages: {info['count']}")
            else:
                print(f"⚠️  WARNING: Topic '{topic_name}' found but type mismatch")
                print(f"   - Expected: {expected_type}")
                print(f"   - Found: {info['type']}")
                validation_passed = False

print("\n" + "="*60)
if validation_passed:
    print("✅ Phase 1 PASSED: All required topics found")
else:
    print("❌ Phase 1 FAILED: Missing or incorrect topics")
    raise ValueError("Cannot proceed without required Cartographer topics")
print("="*60)

### 5.2 Parse and Validate Trajectory Nodes

**Cartographer Message Format:** The `/trajectory_node_list` topic uses `visualization_msgs/MarkerArray` for RViz visualization. Each trajectory node is represented as a SPHERE marker.

**Extraction Strategy:**
- Marker namespace: "Trajectory <trajectory_id>" (e.g., "Trajectory 0")
- Marker ID: Encodes the node ID
- Marker pose: Contains (x, y, z) position and orientation quaternion
- We need to parse these markers to extract structured trajectory data

In [ ]:
print("\n" + "="*60)
print("CARTOGRAPHER DATA VALIDATION - PHASE 2: TRAJECTORY NODES")
print("="*60)

trajectory_nodes = []
trajectory_node_timestamps = []  # Store message receipt times

with Reader(bag_path) as reader:
    print("\nParsing trajectory node messages...")

    for connection, timestamp, rawdata in tqdm(reader.messages()):
        if connection.topic == '/trajectory_node_list':
            try:
                # Deserialize MarkerArray message
                msg = typestore.deserialize_ros1(rawdata, connection.msgtype)

                # Extract markers
                for marker in msg.markers:
                    # Trajectory nodes have namespace like "Trajectory 0"
                    if marker.ns.startswith("Trajectory"):
                        # Extract trajectory ID from namespace
                        traj_id = int(marker.ns.split()[-1])

                        # Node ID is encoded in marker.id
                        node_id = marker.id

                        # Extract pose
                        pose = {
                            'x': marker.pose.position.x,
                            'y': marker.pose.position.y,
                            'z': marker.pose.position.z,
                            'qx': marker.pose.orientation.x,
                            'qy': marker.pose.orientation.y,
                            'qz': marker.pose.orientation.z,
                            'qw': marker.pose.orientation.w
                        }

                        # Store node information
                        # Note: We use message timestamp as proxy for node timestamp
                        trajectory_nodes.append({
                            'node_id': node_id,
                            'trajectory_id': traj_id,
                            'timestamp': timestamp * 1e-9,  # Convert to seconds
                            'pose': pose
                        })

                        trajectory_node_timestamps.append(timestamp * 1e-9)

            except Exception as e:
                print(f"⚠️  Warning: Failed to parse trajectory node message: {e}")
                continue

# Remove duplicates (Cartographer may republish the same node multiple times)
# Keep the latest timestamp for each unique node_id
unique_nodes = {}
for node in trajectory_nodes:
    node_id = node['node_id']
    if node_id not in unique_nodes or node['timestamp'] > unique_nodes[node_id]['timestamp']:
        unique_nodes[node_id] = node

trajectory_nodes = list(unique_nodes.values())
trajectory_nodes.sort(key=lambda n: n['timestamp'])

print(f"\n✅ Extracted {len(trajectory_nodes)} unique trajectory nodes")

# Validation checks
print("\n" + "-"*60)
print("TRAJECTORY NODE VALIDATION CHECKS")
print("-"*60)

validation_results = []

# Check 1: Minimum number of nodes
min_nodes = 50
check_1 = len(trajectory_nodes) >= min_nodes
validation_results.append(('Sufficient nodes (>= 50)', check_1))
print(f"{'✅' if check_1 else '❌'} Node count: {len(trajectory_nodes)} (minimum: {min_nodes})")

# Check 2: Temporal consistency
if len(trajectory_nodes) > 1:
    timestamps = [n['timestamp'] for n in trajectory_nodes]
    time_range = max(timestamps) - min(timestamps)
    check_2 = time_range > 60  # At least 1 minute of trajectory
    validation_results.append(('Temporal span (> 60s)', check_2))
    print(f"{'✅' if check_2 else '❌'} Time range: {time_range:.2f} seconds")
else:
    check_2 = False
    validation_results.append(('Temporal span (> 60s)', check_2))
    print(f"❌ Insufficient nodes for temporal analysis")

# Check 3: Spatial consistency
if len(trajectory_nodes) > 1:
    x_coords = [n['pose']['x'] for n in trajectory_nodes]
    y_coords = [n['pose']['y'] for n in trajectory_nodes]

    map_width = max(x_coords) - min(x_coords)
    map_height = max(y_coords) - min(y_coords)

    check_3 = map_width > 1.0 and map_height > 1.0  # At least 1m x 1m map
    validation_results.append(('Spatial extent (> 1m x 1m)', check_3))
    print(f"{'✅' if check_3 else '❌'} Map dimensions: {map_width:.2f}m x {map_height:.2f}m")
else:
    check_3 = False
    validation_results.append(('Spatial extent (> 1m x 1m)', check_3))
    print(f"❌ Insufficient nodes for spatial analysis")

# Check 4: Node ID consistency
node_ids = [n['node_id'] for n in trajectory_nodes]
unique_node_ids = len(set(node_ids))
check_4 = unique_node_ids == len(trajectory_nodes)
validation_results.append(('Unique node IDs', check_4))
print(f"{'✅' if check_4 else '⚠️ '} Unique node IDs: {unique_node_ids}/{len(trajectory_nodes)}")

# Overall phase result
phase_2_passed = all(result[1] for result in validation_results)

print("\n" + "="*60)
if phase_2_passed:
    print("✅ Phase 2 PASSED: Trajectory nodes are valid")
else:
    print("❌ Phase 2 FAILED: Trajectory node validation failed")
    print("⚠️  Warning: Proceeding may result in poor dataset quality")
print("="*60)

### 5.3 Parse and Validate Constraints (Loop Closures)

**Cartographer Message Format:** The `/constraint_list` topic uses `visualization_msgs/MarkerArray` with LINE_STRIP markers connecting pairs of nodes.

**Constraint Types:**
- **INTRA_SUBMAP:** Constraints within the same local submap (odometry-based)
- **INTER_SUBMAP:** Constraints between different submaps (loop closures) ← **THIS IS WHAT WE NEED**

**Extraction Strategy:**
- Namespace identifies constraint type: "Inter constraints" or "Intra constraints"
- LINE_STRIP markers have exactly 2 points (start and end node)
- Marker ID can be used to link to node pairs
- We extract start/end poses and compute spatial relationship

In [ ]:
print("\n" + "="*60)
print("CARTOGRAPHER DATA VALIDATION - PHASE 3: CONSTRAINTS")
print("="*60)

all_constraints = []
inter_submap_constraints = []
intra_submap_constraints = []

with Reader(bag_path) as reader:
    print("\nParsing constraint messages...")

    for connection, timestamp, rawdata in tqdm(reader.messages()):
        if connection.topic == '/constraint_list':
            try:
                # Deserialize MarkerArray message
                msg = typestore.deserialize_ros1(rawdata, connection.msgtype)

                # Extract markers
                for marker in msg.markers:
                    # Constraints are LINE_STRIP markers
                    if marker.type == 4:  # LINE_STRIP = 4
                        # Determine constraint type from namespace
                        is_inter_submap = "Inter" in marker.ns or "inter" in marker.ns
                        is_intra_submap = "Intra" in marker.ns or "intra" in marker.ns

                        if not (is_inter_submap or is_intra_submap):
                            continue

                        # LINE_STRIP should have exactly 2 points (start and end node)
                        if len(marker.points) == 2:
                            start_point = marker.points[0]
                            end_point = marker.points[1]

                            # Compute spatial distance
                            dx = end_point.x - start_point.x
                            dy = end_point.y - start_point.y
                            dz = end_point.z - start_point.z
                            spatial_distance = np.sqrt(dx**2 + dy**2 + dz**2)

                            constraint = {
                                'constraint_id': marker.id,
                                'type': 'INTER_SUBMAP' if is_inter_submap else 'INTRA_SUBMAP',
                                'start_pose': {
                                    'x': start_point.x,
                                    'y': start_point.y,
                                    'z': start_point.z
                                },
                                'end_pose': {
                                    'x': end_point.x,
                                    'y': end_point.y,
                                    'z': end_point.z
                                },
                                'spatial_distance': spatial_distance,
                                'timestamp': timestamp * 1e-9
                            }

                            all_constraints.append(constraint)

                            if is_inter_submap:
                                inter_submap_constraints.append(constraint)
                            else:
                                intra_submap_constraints.append(constraint)

            except Exception as e:
                print(f"⚠️  Warning: Failed to parse constraint message: {e}")
                continue

print(f"\n✅ Extracted {len(all_constraints)} total constraints")
print(f"   - INTER_SUBMAP (loop closures): {len(inter_submap_constraints)}")
print(f"   - INTRA_SUBMAP (odometry): {len(intra_submap_constraints)}")

# Validation checks
print("\n" + "-"*60)
print("CONSTRAINT VALIDATION CHECKS")
print("-"*60)

validation_results = []

# Check 1: Minimum INTER_SUBMAP constraints
min_inter_constraints = 10
check_1 = len(inter_submap_constraints) >= min_inter_constraints
validation_results.append(('Sufficient INTER_SUBMAP (>= 10)', check_1))
print(f"{'✅' if check_1 else '❌'} INTER_SUBMAP count: {len(inter_submap_constraints)} (minimum: {min_inter_constraints})")

if not check_1:
    print("   ⚠️  WARNING: Insufficient loop closures for robust training")
    print("   → This session may not have enough revisited locations")
    print("   → Consider: (1) longer data collection, (2) more loops, (3) combining multiple sessions")

# Check 2: Spatial distribution of INTER_SUBMAP constraints
if len(inter_submap_constraints) > 0:
    inter_distances = [c['spatial_distance'] for c in inter_submap_constraints]
    mean_distance = np.mean(inter_distances)

    # Loop closures should be spatially close (< 5m typically)
    check_2 = mean_distance < 5.0
    validation_results.append(('Reasonable spatial distances (< 5m)', check_2))
    print(f"{'✅' if check_2 else '⚠️ '} Mean INTER_SUBMAP distance: {mean_distance:.2f}m")
    print(f"   Range: [{min(inter_distances):.2f}, {max(inter_distances):.2f}]m")
else:
    check_2 = False
    validation_results.append(('Reasonable spatial distances (< 5m)', check_2))

# Check 3: Ratio of INTER to INTRA constraints
if len(intra_submap_constraints) > 0:
    inter_intra_ratio = len(inter_submap_constraints) / len(intra_submap_constraints)

    # Typical ratio: 5-20% INTER constraints
    check_3 = 0.05 <= inter_intra_ratio <= 0.30
    validation_results.append(('Reasonable INTER/INTRA ratio (5-30%)', check_3))
    print(f"{'✅' if check_3 else '⚠️ '} INTER/INTRA ratio: {inter_intra_ratio:.2%}")
else:
    check_3 = True  # No INTRA constraints is unusual but not fatal
    validation_results.append(('Reasonable INTER/INTRA ratio (5-30%)', check_3))
    print(f"⚠️  No INTRA constraints found (unusual but not critical)")

# Overall phase result
phase_3_passed = validation_results[0][1]  # Only first check is critical

print("\n" + "="*60)
if phase_3_passed:
    print("✅ Phase 3 PASSED: Constraints are valid")
else:
    print("❌ Phase 3 FAILED: Insufficient INTER_SUBMAP constraints")
    print("⚠️  Cannot proceed with dataset generation")
    raise ValueError("Insufficient loop closure constraints for training")
print("="*60)

### 5.4 Cross-Validation: Match Constraints to Nodes

**Critical Check:** Verify that constraint poses correspond to actual trajectory nodes. This ensures we can later match constraints to feature vectors.

In [ ]:
print("\n" + "="*60)
print("CARTOGRAPHER DATA VALIDATION - PHASE 4: CROSS-VALIDATION")
print("="*60)

# Build spatial index for trajectory nodes
node_positions = np.array([[n['pose']['x'], n['pose']['y'], n['pose']['z']] for n in trajectory_nodes])
node_ids = [n['node_id'] for n in trajectory_nodes]
node_tree = KDTree(node_positions)

print("\nMatching constraints to trajectory nodes...")

matched_constraints = []
unmatched_constraints = []
match_threshold = 0.5  # 50cm tolerance for spatial matching

for constraint in tqdm(inter_submap_constraints):
    # Find nearest node to start pose
    start_pos = np.array([constraint['start_pose']['x'],
                          constraint['start_pose']['y'],
                          constraint['start_pose']['z']])

    start_dist, start_idx = node_tree.query(start_pos)

    # Find nearest node to end pose
    end_pos = np.array([constraint['end_pose']['x'],
                        constraint['end_pose']['y'],
                        constraint['end_pose']['z']])

    end_dist, end_idx = node_tree.query(end_pos)

    # Check if both poses match nodes within threshold
    if start_dist < match_threshold and end_dist < match_threshold:
        # Successfully matched!
        matched_constraint = constraint.copy()
        matched_constraint['node_id_i'] = node_ids[start_idx]
        matched_constraint['node_id_j'] = node_ids[end_idx]
        matched_constraint['match_error_i'] = start_dist
        matched_constraint['match_error_j'] = end_dist

        matched_constraints.append(matched_constraint)
    else:
        unmatched_constraints.append(constraint)

match_rate = len(matched_constraints) / len(inter_submap_constraints) if len(inter_submap_constraints) > 0 else 0

print(f"\n✅ Matched {len(matched_constraints)}/{len(inter_submap_constraints)} constraints to nodes ({match_rate:.1%})")

if len(unmatched_constraints) > 0:
    print(f"⚠️  {len(unmatched_constraints)} constraints could not be matched (tolerance: {match_threshold}m)")

# Validation check
print("\n" + "-"*60)
print("CROSS-VALIDATION CHECKS")
print("-"*60)

# Check: High match rate
min_match_rate = 0.70
check_match_rate = match_rate >= min_match_rate
print(f"{'✅' if check_match_rate else '⚠️ '} Constraint-to-node match rate: {match_rate:.1%} (minimum: {min_match_rate:.0%})")

if not check_match_rate:
    print("   ⚠️  Low match rate may indicate:")
    print("      - Trajectory nodes are incomplete")
    print("      - Constraint poses don't align with node poses")
    print("      - Need to adjust match_threshold")

# Check: Sufficient matched constraints for training
min_matched_constraints = 10
check_sufficient = len(matched_constraints) >= min_matched_constraints
print(f"{'✅' if check_sufficient else '❌'} Matched constraint count: {len(matched_constraints)} (minimum: {min_matched_constraints})")

# Overall phase result
phase_4_passed = check_sufficient

print("\n" + "="*60)
if phase_4_passed:
    print("✅ Phase 4 PASSED: Constraints successfully matched to nodes")
else:
    print("❌ Phase 4 FAILED: Insufficient matched constraints")
    print("⚠️  Cannot proceed with dataset generation")
    raise ValueError("Unable to match constraints to trajectory nodes")
print("="*60)

# Store matched constraints for later use
inter_submap_constraints = matched_constraints

### 5.5 Cartographer Validation Summary

In [ ]:
print("\n" + "="*70)
print("CARTOGRAPHER DATA VALIDATION - FINAL SUMMARY")
print("="*70)

validation_summary = f"""
📊 CARTOGRAPHER DATA VALIDATION REPORT
{'='*70}

SESSION INFORMATION:
  • Session ID: {session_id}
  • Duration: {duration_sec:.2f} seconds ({duration_sec/60:.2f} minutes)
  • Bag size: {file_size_mb:.2f} MB

TRAJECTORY NODES:
  • Total unique nodes: {len(trajectory_nodes)}
  • Time range: {trajectory_nodes[-1]['timestamp'] - trajectory_nodes[0]['timestamp']:.2f} seconds
  • Map dimensions: {map_width:.2f}m x {map_height:.2f}m

CONSTRAINTS:
  • Total constraints: {len(all_constraints)}
  • INTER_SUBMAP (loop closures): {len(matched_constraints)}
  • INTRA_SUBMAP (odometry): {len(intra_submap_constraints)}
  • Match rate (INTER to nodes): {match_rate:.1%}

VALIDATION STATUS:
  ✅ Phase 1: Required topics present
  {'✅' if phase_2_passed else '❌'} Phase 2: Trajectory nodes valid
  {'✅' if phase_3_passed else '❌'} Phase 3: Constraints valid
  {'✅' if phase_4_passed else '❌'} Phase 4: Cross-validation passed

DATASET GENERATION READINESS:
  {'✅' if len(matched_constraints) >= 20 else '⚠️ '} Positive pairs available: {len(matched_constraints)} (target: >= 20)
  {'✅' if len(trajectory_nodes) >= 100 else '⚠️ '} Nodes for negatives: {len(trajectory_nodes)} (target: >= 100)

{'='*70}
"""

print(validation_summary)

# Save validation report
with open('cartographer_validation_report.txt', 'w') as f:
    f.write(validation_summary)

print("✅ Validation report saved to: cartographer_validation_report.txt")

# Overall validation result
overall_passed = phase_2_passed and phase_3_passed and phase_4_passed

print("\n" + "="*70)
if overall_passed:
    print("🎉 CARTOGRAPHER DATA VALIDATION: PASSED")
    print("   → Ready to proceed with dataset generation")
else:
    print("❌ CARTOGRAPHER DATA VALIDATION: FAILED")
    print("   → Cannot proceed with dataset generation")
    print("   → Review validation report and address issues")
print("="*70)

## <font color='#2E86AB'>▼ 6. Load Extracted Features</font>

Load the pre-extracted camera and LiDAR features from the HDF5 file. These features were generated independently and need to be aligned to trajectory nodes.

In [ ]:
print("=" * 60)
print("LOADING EXTRACTED FEATURES")
print("=" * 60)

# Load features from HDF5
with h5py.File(features_path, 'r') as f:
    # Load camera features
    camera_features = f['camera/features'][:]
    camera_timestamps = f['camera/timestamps'][:]
    camera_filenames = [fn.decode('utf-8') if isinstance(fn, bytes) else fn for fn in f['camera/filenames'][:]]

    # Load LiDAR features
    lidar_features = f['lidar/features'][:]
    lidar_timestamps = f['lidar/timestamps'][:]
    lidar_filenames = [fn.decode('utf-8') if isinstance(fn, bytes) else fn for fn in f['lidar/filenames'][:]]

print(f"\n✅ Features loaded successfully")
print(f"\nCamera features:")
print(f"  Shape: {camera_features.shape}")
print(f"  Timestamps: {len(camera_timestamps)}")
print(f"  Time range: [{camera_timestamps[0]:.2f}, {camera_timestamps[-1]:.2f}]")

print(f"\nLiDAR features:")
print(f"  Shape: {lidar_features.shape}")
print(f"  Timestamps: {len(lidar_timestamps)}")
print(f"  Time range: [{lidar_timestamps[0]:.2f}, {lidar_timestamps[-1]:.2f}]")

# Validate feature properties
print("\n" + "-" * 60)
print("FEATURE VALIDATION")
print("-" * 60)

# Check L2 normalization
camera_norms = np.linalg.norm(camera_features, axis=1)
lidar_norms = np.linalg.norm(lidar_features, axis=1)

camera_normalized = np.allclose(camera_norms, 1.0, atol=1e-5)
lidar_normalized = np.allclose(lidar_norms, 1.0, atol=1e-5)

print(f"{'✅' if camera_normalized else '❌'} Camera features L2 normalized: {camera_normalized}")
print(f"   Mean norm: {np.mean(camera_norms):.6f}, Std: {np.std(camera_norms):.6f}")

print(f"{'✅' if lidar_normalized else '❌'} LiDAR features L2 normalized: {lidar_normalized}")
print(f"   Mean norm: {np.mean(lidar_norms):.6f}, Std: {np.std(lidar_norms):.6f}")

# Check dimensions
expected_camera_dim = 1280
expected_lidar_dim = 256

camera_dim_ok = camera_features.shape[1] == expected_camera_dim
lidar_dim_ok = lidar_features.shape[1] == expected_lidar_dim

print(f"{'✅' if camera_dim_ok else '❌'} Camera feature dimension: {camera_features.shape[1]} (expected: {expected_camera_dim})")
print(f"{'✅' if lidar_dim_ok else '❌'} LiDAR feature dimension: {lidar_features.shape[1]} (expected: {expected_lidar_dim})")

if not (camera_normalized and lidar_normalized and camera_dim_ok and lidar_dim_ok):
    raise ValueError("Feature validation failed - check features.h5 file")

print("\n✅ All feature validations passed")

## <font color='#2E86AB'>▼ 7. Time Alignment: Features ↔ Trajectory Nodes</font>

**Challenge:** Features and trajectory nodes have different timestamps due to asynchronous multi-rate sensors.

**Solution:** Bidirectional nearest neighbor matching with temporal tolerance (0.5 seconds).

**Goal:** Create a unified database where each valid trajectory node has both camera and LiDAR features.

### 7.1 Align Camera Features to Nodes

In [ ]:
def align_features_to_nodes(feature_timestamps, trajectory_nodes, max_offset=0.5):
    """
    Align features to trajectory nodes using nearest neighbor matching.

    Args:
        feature_timestamps: Array of feature timestamps (seconds)
        trajectory_nodes: List of trajectory node dictionaries
        max_offset: Maximum allowable time difference (seconds)

    Returns:
        alignment_map: Dict mapping feature_idx -> {'node_id', 'time_offset', 'node_timestamp'}
    """
    # Build KD-tree for fast nearest neighbor search
    node_timestamps = np.array([n['timestamp'] for n in trajectory_nodes])
    node_ids = [n['node_id'] for n in trajectory_nodes]

    tree = KDTree(node_timestamps.reshape(-1, 1))

    alignment_map = {}

    for feat_idx, feat_time in enumerate(feature_timestamps):
        # Find nearest node
        dist, idx = tree.query([[feat_time]], k=1)
        time_diff = abs(dist[0][0])

        if time_diff < max_offset:
            # Valid alignment
            alignment_map[feat_idx] = {
                'node_id': node_ids[idx[0]],
                'time_offset': time_diff,
                'node_timestamp': node_timestamps[idx[0]]
            }

    return alignment_map

print("=" * 60)
print("TIME ALIGNMENT - CAMERA FEATURES")
print("=" * 60)

camera_alignment = align_features_to_nodes(camera_timestamps, trajectory_nodes, max_offset=0.5)

camera_alignment_rate = len(camera_alignment) / len(camera_timestamps)
time_offsets = [a['time_offset'] for a in camera_alignment.values()]

print(f"\n✅ Camera alignment complete")
print(f"   Aligned: {len(camera_alignment)}/{len(camera_timestamps)} ({camera_alignment_rate:.1%})")
print(f"   Mean time offset: {np.mean(time_offsets):.3f}s")
print(f"   Max time offset: {np.max(time_offsets):.3f}s")

### 7.2 Align LiDAR Features to Nodes

In [ ]:
print("=" * 60)
print("TIME ALIGNMENT - LIDAR FEATURES")
print("=" * 60)

lidar_alignment = align_features_to_nodes(lidar_timestamps, trajectory_nodes, max_offset=0.5)

lidar_alignment_rate = len(lidar_alignment) / len(lidar_timestamps)
time_offsets = [a['time_offset'] for a in lidar_alignment.values()]

print(f"\n✅ LiDAR alignment complete")
print(f"   Aligned: {len(lidar_alignment)}/{len(lidar_timestamps)} ({lidar_alignment_rate:.1%})")
print(f"   Mean time offset: {np.mean(time_offsets):.3f}s")
print(f"   Max time offset: {np.max(time_offsets):.3f}s")

### 7.3 Create Unified Node Database

In [ ]:
print("=" * 60)
print("CREATING UNIFIED NODE DATABASE")
print("=" * 60)

# Create database indexed by node_id
node_database = {}

# Initialize with all trajectory nodes
for node in trajectory_nodes:
    node_database[node['node_id']] = {
        'node_id': node['node_id'],
        'trajectory_id': node['trajectory_id'],
        'timestamp': node['timestamp'],
        'pose': node['pose'],
        'camera_feature': None,
        'lidar_feature': None,
        'has_camera': False,
        'has_lidar': False,
        'has_both': False
    }

# Add camera features
for feat_idx, alignment_info in camera_alignment.items():
    node_id = alignment_info['node_id']
    if node_id in node_database:
        node_database[node_id]['camera_feature'] = camera_features[feat_idx]
        node_database[node_id]['has_camera'] = True

# Add LiDAR features
for feat_idx, alignment_info in lidar_alignment.items():
    node_id = alignment_info['node_id']
    if node_id in node_database:
        node_database[node_id]['lidar_feature'] = lidar_features[feat_idx]
        node_database[node_id]['has_lidar'] = True

# Mark nodes with both modalities
for node_id, node_data in node_database.items():
    node_data['has_both'] = node_data['has_camera'] and node_data['has_lidar']

    # Concatenate features if both available
    if node_data['has_both']:
        node_data['combined_feature'] = np.concatenate([
            node_data['camera_feature'],
            node_data['lidar_feature']
        ])

# Filter to valid nodes (both modalities)
valid_nodes = {k: v for k, v in node_database.items() if v['has_both']}

print(f"\n✅ Unified database created")
print(f"   Total nodes: {len(node_database)}")
print(f"   Nodes with camera: {sum(1 for n in node_database.values() if n['has_camera'])}")
print(f"   Nodes with LiDAR: {sum(1 for n in node_database.values() if n['has_lidar'])}")
print(f"   Nodes with both: {len(valid_nodes)}")
print(f"   Combined feature dimension: {valid_nodes[list(valid_nodes.keys())[0]]['combined_feature'].shape[0]}D")

# Validation check
min_valid_nodes = 50
if len(valid_nodes) < min_valid_nodes:
    print(f"\n❌ Insufficient valid nodes: {len(valid_nodes)} (minimum: {min_valid_nodes})")
    raise ValueError("Not enough nodes with both camera and LiDAR features")
else:
    print(f"\n✅ Sufficient valid nodes for dataset generation")

## <font color='#2E86AB'>▼ 8. Generate Positive Pairs (Loop Closures)</font>

**Source:** INTER_SUBMAP constraints from Cartographer (validated in Section 5)

**Criteria:**
- Both nodes must be in valid_nodes (have both modalities)
- Spatial distance < 2.0m
- Angular distance < π/2 radians

**Label:** 1 (positive loop closure)

In [ ]:
def quaternion_to_yaw(qx, qy, qz, qw):
    """Convert quaternion to yaw angle (rotation around z-axis)."""
    siny_cosp = 2 * (qw * qz + qx * qy)
    cosy_cosp = 1 - 2 * (qy * qy + qz * qz)
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    return yaw

def compute_angular_distance(yaw1, yaw2):
    """Compute angular distance between two yaw angles (0 to π)."""
    diff = abs(yaw1 - yaw2)
    # Normalize to [0, π]
    if diff > np.pi:
        diff = 2 * np.pi - diff
    return diff

print("=" * 60)
print("GENERATING POSITIVE PAIRS")
print("=" * 60)

positive_pairs = []
max_spatial_distance = 2.0  # meters
max_angular_distance = np.pi / 2  # 90 degrees

print(f"\nProcessing {len(inter_submap_constraints)} INTER_SUBMAP constraints...")

for constraint in tqdm(inter_submap_constraints):
    node_id_i = constraint['node_id_i']
    node_id_j = constraint['node_id_j']

    # Check if both nodes are valid
    if node_id_i not in valid_nodes or node_id_j not in valid_nodes:
        continue

    # Get node data
    node_i = valid_nodes[node_id_i]
    node_j = valid_nodes[node_id_j]

    # Compute spatial distance
    dx = node_i['pose']['x'] - node_j['pose']['x']
    dy = node_i['pose']['y'] - node_j['pose']['y']
    spatial_distance = np.sqrt(dx**2 + dy**2)

    # Compute angular distance
    yaw_i = quaternion_to_yaw(node_i['pose']['qx'], node_i['pose']['qy'],
                              node_i['pose']['qz'], node_i['pose']['qw'])
    yaw_j = quaternion_to_yaw(node_j['pose']['qx'], node_j['pose']['qy'],
                              node_j['pose']['qz'], node_j['pose']['qw'])
    angular_distance = compute_angular_distance(yaw_i, yaw_j)

    # Apply thresholds
    if spatial_distance < max_spatial_distance and angular_distance < max_angular_distance:
        positive_pairs.append({
            'query_node_id': node_id_i,
            'candidate_node_id': node_id_j,
            'label': 1,
            'pair_type': 'positive',
            'spatial_distance': spatial_distance,
            'angular_distance': angular_distance,
            'temporal_distance': abs(node_i['timestamp'] - node_j['timestamp'])
        })

print(f"\n✅ Generated {len(positive_pairs)} positive pairs")

if len(positive_pairs) > 0:
    spatial_dists = [p['spatial_distance'] for p in positive_pairs]
    angular_dists = [p['angular_distance'] for p in positive_pairs]
    temporal_dists = [p['temporal_distance'] for p in positive_pairs]

    print(f"\nSpatial distance statistics:")
    print(f"   Mean: {np.mean(spatial_dists):.2f}m")
    print(f"   Range: [{np.min(spatial_dists):.2f}, {np.max(spatial_dists):.2f}]m")

    print(f"\nAngular distance statistics:")
    print(f"   Mean: {np.rad2deg(np.mean(angular_dists)):.1f}°")
    print(f"   Range: [{np.rad2deg(np.min(angular_dists)):.1f}°, {np.rad2deg(np.max(angular_dists)):.1f}°]")

    print(f"\nTemporal distance statistics:")
    print(f"   Mean: {np.mean(temporal_dists):.2f}s")
    print(f"   Range: [{np.min(temporal_dists):.2f}, {np.max(temporal_dists):.2f}]s")
else:
    print("\n❌ No positive pairs generated - check thresholds or constraints")
    raise ValueError("Cannot proceed without positive pairs")

## <font color='#2E86AB'>▼ 9. Generate Easy Negative Pairs</font>

**Strategy:** Random sampling from spatially and temporally distant locations

**Criteria:**
- Spatial distance > 5.0m (clearly different locations)
- Temporal distance > 5.0s (not consecutive frames)

**Target:** 35% of total dataset (1.17× positive pairs)

**Label:** 0 (not a loop closure)

In [ ]:
print("=" * 60)
print("GENERATING EASY NEGATIVE PAIRS")
print("=" * 60)

min_spatial_distance = 5.0  # meters
min_temporal_distance = 5.0  # seconds

# Calculate target number (35% of total, so 35/30 * num_positive)
num_target = int(len(positive_pairs) * 35 / 30)

print(f"\nTarget: {num_target} easy negative pairs")
print(f"Criteria: spatial > {min_spatial_distance}m, temporal > {min_temporal_distance}s")

easy_negative_pairs = []
valid_node_ids = list(valid_nodes.keys())
max_attempts = num_target * 20  # Prevent infinite loop
attempts = 0

print("\nSampling pairs...")
with tqdm(total=num_target) as pbar:
    while len(easy_negative_pairs) < num_target and attempts < max_attempts:
        attempts += 1

        # Random sample two different nodes
        node_id_i, node_id_j = np.random.choice(valid_node_ids, size=2, replace=False)

        node_i = valid_nodes[node_id_i]
        node_j = valid_nodes[node_id_j]

        # Compute spatial distance
        dx = node_i['pose']['x'] - node_j['pose']['x']
        dy = node_i['pose']['y'] - node_j['pose']['y']
        spatial_distance = np.sqrt(dx**2 + dy**2)

        # Compute temporal distance
        temporal_distance = abs(node_i['timestamp'] - node_j['timestamp'])

        # Check criteria
        if spatial_distance > min_spatial_distance and temporal_distance > min_temporal_distance:
            # Check for duplicates (avoid same pair in reverse order)
            pair_key = tuple(sorted([node_id_i, node_id_j]))
            existing_keys = [tuple(sorted([p['query_node_id'], p['candidate_node_id']]))
                           for p in easy_negative_pairs]

            if pair_key not in existing_keys:
                easy_negative_pairs.append({
                    'query_node_id': node_id_i,
                    'candidate_node_id': node_id_j,
                    'label': 0,
                    'pair_type': 'easy_negative',
                    'spatial_distance': spatial_distance,
                    'temporal_distance': temporal_distance
                })
                pbar.update(1)

if len(easy_negative_pairs) < num_target:
    print(f"\n⚠️  Warning: Only generated {len(easy_negative_pairs)}/{num_target} easy negatives")
    print("   (May need to relax thresholds or collect more data)")
else:
    print(f"\n✅ Generated {len(easy_negative_pairs)} easy negative pairs")

if len(easy_negative_pairs) > 0:
    spatial_dists = [p['spatial_distance'] for p in easy_negative_pairs]
    temporal_dists = [p['temporal_distance'] for p in easy_negative_pairs]

    print(f"\nSpatial distance statistics:")
    print(f"   Mean: {np.mean(spatial_dists):.2f}m")
    print(f"   Range: [{np.min(spatial_dists):.2f}, {np.max(spatial_dists):.2f}]m")

    print(f"\nTemporal distance statistics:")
    print(f"   Mean: {np.mean(temporal_dists):.2f}s")
    print(f"   Range: [{np.min(temporal_dists):.2f}, {np.max(temporal_dists):.2f}]s")

## <font color='#2E86AB'>▼ 10. Generate Hard Negative Pairs</font>

**Critical for robustness:** Hard negatives teach the model to distinguish challenging cases.

**Two types:**
- **Type A (Perceptual Aliasing):** High feature similarity but spatially distant
- **Type B (Different Viewpoints):** Same location but different viewing angle

**Target:** 35% of total dataset (1.17× positive pairs)

**Label:** 0 (not a loop closure despite similarity/proximity)

### 10.1 Type A: Perceptual Aliasing (Similar Features, Different Locations)

In [ ]:
print("=" * 60)
print("GENERATING HARD NEGATIVE PAIRS - TYPE A (PERCEPTUAL ALIASING)")
print("=" * 60)

min_similarity = 0.7  # Cosine similarity threshold
min_spatial_distance = 3.0  # meters (must be spatially distant)

# Calculate target (half of hard negatives budget)
num_hard_total = int(len(positive_pairs) * 35 / 30)
num_target_type_a = num_hard_total // 2

print(f"\nTarget: {num_target_type_a} Type A hard negatives")
print(f"Criteria: cosine similarity > {min_similarity}, spatial > {min_spatial_distance}m")

hard_negative_pairs_type_a = []
valid_node_ids = list(valid_nodes.keys())

# Compute pairwise cosine similarities (this is expensive, so we'll sample)
print("\nComputing feature similarities...")
n_samples = min(len(valid_node_ids), 200)  # Limit computation
sampled_node_ids = np.random.choice(valid_node_ids, size=n_samples, replace=False)

# Build feature matrix
features_matrix = np.array([valid_nodes[nid]['combined_feature'] for nid in sampled_node_ids])

# Compute all pairwise cosine similarities
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(features_matrix)

print("\nSearching for perceptually similar pairs...")
max_attempts = num_target_type_a * 20
attempts = 0

with tqdm(total=num_target_type_a) as pbar:
    while len(hard_negative_pairs_type_a) < num_target_type_a and attempts < max_attempts:
        attempts += 1

        # Random sample from high-similarity pairs
        i, j = np.random.randint(0, n_samples, size=2)
        if i == j:
            continue

        similarity = similarity_matrix[i, j]

        if similarity > min_similarity:
            node_id_i = sampled_node_ids[i]
            node_id_j = sampled_node_ids[j]

            node_i = valid_nodes[node_id_i]
            node_j = valid_nodes[node_id_j]

            # Compute spatial distance
            dx = node_i['pose']['x'] - node_j['pose']['x']
            dy = node_i['pose']['y'] - node_j['pose']['y']
            spatial_distance = np.sqrt(dx**2 + dy**2)

            if spatial_distance > min_spatial_distance:
                # Check for duplicates
                pair_key = tuple(sorted([node_id_i, node_id_j]))
                existing_keys = [tuple(sorted([p['query_node_id'], p['candidate_node_id']]))
                               for p in hard_negative_pairs_type_a]

                if pair_key not in existing_keys:
                    hard_negative_pairs_type_a.append({
                        'query_node_id': node_id_i,
                        'candidate_node_id': node_id_j,
                        'label': 0,
                        'pair_type': 'hard_negative_type_a',
                        'spatial_distance': spatial_distance,
                        'feature_similarity': similarity
                    })
                    pbar.update(1)

print(f"\n✅ Generated {len(hard_negative_pairs_type_a)} Type A hard negatives")

if len(hard_negative_pairs_type_a) > 0:
    spatial_dists = [p['spatial_distance'] for p in hard_negative_pairs_type_a]
    similarities = [p['feature_similarity'] for p in hard_negative_pairs_type_a]

    print(f"\nSpatial distance statistics:")
    print(f"   Mean: {np.mean(spatial_dists):.2f}m")
    print(f"   Range: [{np.min(spatial_dists):.2f}, {np.max(spatial_dists):.2f}]m")

    print(f"\nFeature similarity statistics:")
    print(f"   Mean: {np.mean(similarities):.3f}")
    print(f"   Range: [{np.min(similarities):.3f}, {np.max(similarities):.3f}]")

### 10.2 Type B: Same Location, Different Viewpoint

In [ ]:
print("=" * 60)
print("GENERATING HARD NEGATIVE PAIRS - TYPE B (DIFFERENT VIEWPOINTS)")
print("=" * 60)

max_spatial_distance = 2.0  # meters (same location)
min_angular_distance = np.pi / 2  # 90 degrees (different viewpoint)
min_temporal_distance = 5.0  # seconds (temporally distinct)

# Calculate target (remaining half of hard negatives budget)
num_target_type_b = num_hard_total - len(hard_negative_pairs_type_a)

print(f"\nTarget: {num_target_type_b} Type B hard negatives")
print(f"Criteria: spatial < {max_spatial_distance}m, angular > {np.rad2deg(min_angular_distance):.0f}°, temporal > {min_temporal_distance}s")

hard_negative_pairs_type_b = []
max_attempts = num_target_type_b * 20
attempts = 0

print("\nSearching for same-location different-viewpoint pairs...")
with tqdm(total=num_target_type_b) as pbar:
    while len(hard_negative_pairs_type_b) < num_target_type_b and attempts < max_attempts:
        attempts += 1

        # Random sample two different nodes
        node_id_i, node_id_j = np.random.choice(valid_node_ids, size=2, replace=False)

        node_i = valid_nodes[node_id_i]
        node_j = valid_nodes[node_id_j]

        # Compute spatial distance
        dx = node_i['pose']['x'] - node_j['pose']['x']
        dy = node_i['pose']['y'] - node_j['pose']['y']
        spatial_distance = np.sqrt(dx**2 + dy**2)

        # Compute angular distance
        yaw_i = quaternion_to_yaw(node_i['pose']['qx'], node_i['pose']['qy'],
                                  node_i['pose']['qz'], node_i['pose']['qw'])
        yaw_j = quaternion_to_yaw(node_j['pose']['qx'], node_j['pose']['qy'],
                                  node_j['pose']['qz'], node_j['pose']['qw'])
        angular_distance = compute_angular_distance(yaw_i, yaw_j)

        # Compute temporal distance
        temporal_distance = abs(node_i['timestamp'] - node_j['timestamp'])

        # Check criteria
        if (spatial_distance < max_spatial_distance and
            angular_distance > min_angular_distance and
            temporal_distance > min_temporal_distance):

            # Check for duplicates
            pair_key = tuple(sorted([node_id_i, node_id_j]))
            existing_keys = [tuple(sorted([p['query_node_id'], p['candidate_node_id']]))
                           for p in hard_negative_pairs_type_b]

            if pair_key not in existing_keys:
                hard_negative_pairs_type_b.append({
                    'query_node_id': node_id_i,
                    'candidate_node_id': node_id_j,
                    'label': 0,
                    'pair_type': 'hard_negative_type_b',
                    'spatial_distance': spatial_distance,
                    'angular_distance': angular_distance,
                    'temporal_distance': temporal_distance
                })
                pbar.update(1)

print(f"\n✅ Generated {len(hard_negative_pairs_type_b)} Type B hard negatives")

if len(hard_negative_pairs_type_b) > 0:
    spatial_dists = [p['spatial_distance'] for p in hard_negative_pairs_type_b]
    angular_dists = [p['angular_distance'] for p in hard_negative_pairs_type_b]
    temporal_dists = [p['temporal_distance'] for p in hard_negative_pairs_type_b]

    print(f"\nSpatial distance statistics:")
    print(f"   Mean: {np.mean(spatial_dists):.2f}m")
    print(f"   Range: [{np.min(spatial_dists):.2f}, {np.max(spatial_dists):.2f}]m")

    print(f"\nAngular distance statistics:")
    print(f"   Mean: {np.rad2deg(np.mean(angular_dists)):.1f}°")
    print(f"   Range: [{np.rad2deg(np.min(angular_dists)):.1f}°, {np.rad2deg(np.max(angular_dists)):.1f}°]")

    print(f"\nTemporal distance statistics:")
    print(f"   Mean: {np.mean(temporal_dists):.2f}s")
    print(f"   Range: [{np.min(temporal_dists):.2f}, {np.max(temporal_dists):.2f}]s")

### 10.3 Combine All Hard Negatives

In [ ]:
# Combine both types of hard negatives
hard_negative_pairs = hard_negative_pairs_type_a + hard_negative_pairs_type_b

print("=" * 60)
print("HARD NEGATIVE PAIRS SUMMARY")
print("=" * 60)
print(f"\nTotal hard negatives: {len(hard_negative_pairs)}")
print(f"   Type A (perceptual aliasing): {len(hard_negative_pairs_type_a)}")
print(f"   Type B (different viewpoint): {len(hard_negative_pairs_type_b)}")

## <font color='#2E86AB'>▼ 11. Compute Pairwise Features & Create Dataset</font>

**Pairwise Encoding:** For each pair (query, candidate), compute:
```
pairwise_feature = abs(f_query - f_candidate)
```

Where:
- `f_query` = concatenated [camera_1280D + lidar_256D] = 1536D
- `f_candidate` = concatenated [camera_1280D + lidar_256D] = 1536D
- Result: 1536D difference vector

**Properties:**
- Symmetric: |a - b| = |b - a|
- Range: [0, √2] for L2-normalized features
- Small values → similar → likely loop closure
- Large values → different → likely not loop closure

In [ ]:
print("=" * 60)
print("COMBINING ALL PAIRS")
print("=" * 60)

# Combine all pairs
all_pairs = positive_pairs + easy_negative_pairs + hard_negative_pairs

print(f"\n📊 Dataset Composition:")
print(f"   Total pairs: {len(all_pairs)}")
print(f"   Positive: {len(positive_pairs)} ({100*len(positive_pairs)/len(all_pairs):.1f}%)")
print(f"   Easy negative: {len(easy_negative_pairs)} ({100*len(easy_negative_pairs)/len(all_pairs):.1f}%)")
print(f"   Hard negative: {len(hard_negative_pairs)} ({100*len(hard_negative_pairs)/len(all_pairs):.1f}%)")
print(f"      • Type A: {len(hard_negative_pairs_type_a)} ({100*len(hard_negative_pairs_type_a)/len(all_pairs):.1f}%)")
print(f"      • Type B: {len(hard_negative_pairs_type_b)} ({100*len(hard_negative_pairs_type_b)/len(all_pairs):.1f}%)")

# Validate label balance
positive_ratio = len(positive_pairs) / len(all_pairs)
target_positive_ratio = 0.30

if 0.25 <= positive_ratio <= 0.35:
    print(f"\n✅ Label balance good: {positive_ratio:.1%} positive (target: {target_positive_ratio:.0%})")
else:
    print(f"\n⚠️  Label balance suboptimal: {positive_ratio:.1%} positive (target: {target_positive_ratio:.0%})")

In [ ]:
print("\n" + "=" * 60)
print("COMPUTING PAIRWISE FEATURES")
print("=" * 60)

dataset = []

print(f"\nProcessing {len(all_pairs)} pairs...")
for pair in tqdm(all_pairs):
    query_node_id = pair['query_node_id']
    candidate_node_id = pair['candidate_node_id']

    # Get combined features (1536D each)
    f_query = valid_nodes[query_node_id]['combined_feature']
    f_candidate = valid_nodes[candidate_node_id]['combined_feature']

    # Compute pairwise feature: absolute difference
    pairwise_feature = np.abs(f_query - f_candidate)

    # Create dataset entry
    dataset.append({
        'pairwise_feature': pairwise_feature,
        'label': pair['label'],
        'query_node_id': query_node_id,
        'candidate_node_id': candidate_node_id,
        'pair_type': pair['pair_type'],
        'query_timestamp': valid_nodes[query_node_id]['timestamp'],
        'candidate_timestamp': valid_nodes[candidate_node_id]['timestamp'],
        'spatial_distance': pair.get('spatial_distance', None),
        'metadata': pair  # Keep all original pair info
    })

print(f"\n✅ Pairwise features computed for {len(dataset)} pairs")

# Convert to arrays for analysis
X = np.array([d['pairwise_feature'] for d in dataset])
y = np.array([d['label'] for d in dataset])

print(f"\n📊 Dataset Arrays:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"   Positive samples: {np.sum(y == 1)} ({100*np.mean(y):.1f}%)")
print(f"   Negative samples: {np.sum(y == 0)} ({100*(1-np.mean(y)):.1f}%)")

# Feature statistics
print(f"\n📈 Pairwise Feature Statistics:")
print(f"   Mean: {np.mean(X):.4f}")
print(f"   Std: {np.std(X):.4f}")
print(f"   Min: {np.min(X):.4f}")
print(f"   Max: {np.max(X):.4f}")
print(f"   Theoretical max: {np.sqrt(2):.4f} (√2 for L2-normalized features)")

## <font color='#2E86AB'>▼ 12. Train/Validation/Test Split</font>

**Strategy:** Temporal splitting to prevent data leakage

**Rationale:** The model should generalize to future trajectory segments, not memorize past ones. Splitting by time ensures test data represents unseen temporal regions.

**Split:**
- Train: First 60% of trajectory
- Validation: Middle 20% of trajectory
- Test: Last 20% of trajectory

**Assignment:** Each pair is assigned to a split based on the LATER timestamp (max of query and candidate).

In [ ]:
print("=" * 60)
print("TRAIN/VALIDATION/TEST SPLIT (TEMPORAL)")
print("=" * 60)

# Get all timestamps from valid nodes
all_timestamps = sorted([node['timestamp'] for node in valid_nodes.values()])
min_time = all_timestamps[0]
max_time = all_timestamps[-1]
time_range = max_time - min_time

# Define split boundaries
train_end = min_time + 0.60 * time_range
val_end = min_time + 0.80 * time_range

print(f"\nTime range: [{min_time:.2f}, {max_time:.2f}] seconds")
print(f"Duration: {time_range:.2f} seconds")
print(f"\nSplit boundaries:")
print(f"   Train: [{min_time:.2f}, {train_end:.2f}]")
print(f"   Val:   [{train_end:.2f}, {val_end:.2f}]")
print(f"   Test:  [{val_end:.2f}, {max_time:.2f}]")

# Assign each pair to a split based on the LATER timestamp
train_dataset = []
val_dataset = []
test_dataset = []

print("\nAssigning pairs to splits...")
for data_point in tqdm(dataset):
    # Use the maximum timestamp (later observation)
    pair_time = max(data_point['query_timestamp'], data_point['candidate_timestamp'])

    if pair_time < train_end:
        train_dataset.append(data_point)
    elif pair_time < val_end:
        val_dataset.append(data_point)
    else:
        test_dataset.append(data_point)

print(f"\n✅ Split complete")
print(f"\n📊 Split Statistics:")
print(f"   Train: {len(train_dataset)} pairs ({100*len(train_dataset)/len(dataset):.1f}%)")
print(f"   Val:   {len(val_dataset)} pairs ({100*len(val_dataset)/len(dataset):.1f}%)")
print(f"   Test:  {len(test_dataset)} pairs ({100*len(test_dataset)/len(dataset):.1f}%)")

# Check label balance in each split
def get_label_stats(split_data, split_name):
    labels = [d['label'] for d in split_data]
    positive_count = sum(labels)
    negative_count = len(labels) - positive_count
    positive_ratio = positive_count / len(labels) if len(labels) > 0 else 0

    print(f"\n   {split_name}:")
    print(f"      Positive: {positive_count} ({100*positive_ratio:.1f}%)")
    print(f"      Negative: {negative_count} ({100*(1-positive_ratio):.1f}%)")

    return positive_ratio

print("\n📊 Label Balance per Split:")
train_pos_ratio = get_label_stats(train_dataset, "Train")
val_pos_ratio = get_label_stats(val_dataset, "Val")
test_pos_ratio = get_label_stats(test_dataset, "Test")

# Validate minimum samples per split
min_samples = 10
if len(train_dataset) < min_samples or len(val_dataset) < min_samples or len(test_dataset) < min_samples:
    print(f"\n⚠️  Warning: Some splits have fewer than {min_samples} samples")
    print("   Consider collecting more data or adjusting split ratios")

## <font color='#2E86AB'>▼ 13. Dataset Quality Validation & Visualization</font>

Final validation checks and diagnostic visualizations before saving the dataset.

In [ ]:
print("=" * 60)
print("DATASET QUALITY VALIDATION")
print("=" * 60)

validation_checks = []

# Check 1: Sufficient total samples
min_total_samples = 50
check_1 = len(dataset) >= min_total_samples
validation_checks.append(('Sufficient samples (>= 50)', check_1))
print(f"\n{'✅' if check_1 else '❌'} Total samples: {len(dataset)} (minimum: {min_total_samples})")

# Check 2: Label balance
positive_ratio = np.mean(y)
check_2 = 0.20 <= positive_ratio <= 0.40
validation_checks.append(('Label balance (20-40% positive)', check_2))
print(f"{'✅' if check_2 else '⚠️ '} Positive ratio: {positive_ratio:.1%} (target: 25-35%)")

# Check 3: Feature quality
mean_feature = np.mean(X)
std_feature = np.std(X)
check_3 = 0.1 < mean_feature < 1.0 and std_feature > 0.05
validation_checks.append(('Feature statistics reasonable', check_3))
print(f"{'✅' if check_3 else '⚠️ '} Mean: {mean_feature:.3f}, Std: {std_feature:.3f}")

# Check 4: No NaN or Inf values
check_4 = not (np.any(np.isnan(X)) or np.any(np.isinf(X)))
validation_checks.append(('No NaN/Inf values', check_4))
print(f"{'✅' if check_4 else '❌'} Data integrity: {'OK' if check_4 else 'FAILED'}")

# Check 5: All splits have samples
check_5 = len(train_dataset) > 0 and len(val_dataset) > 0 and len(test_dataset) > 0
validation_checks.append(('All splits non-empty', check_5))
print(f"{'✅' if check_5 else '❌'} Splits: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

# Check 6: Hard negatives present
hard_neg_ratio = len(hard_negative_pairs) / len(all_pairs)
check_6 = hard_neg_ratio >= 0.20
validation_checks.append(('Sufficient hard negatives (>= 20%)', check_6))
print(f"{'✅' if check_6 else '⚠️ '} Hard negative ratio: {hard_neg_ratio:.1%}")

# Overall validation
all_passed = all(check[1] for check in validation_checks)
critical_passed = validation_checks[0][1] and validation_checks[3][1] and validation_checks[4][1]

print("\n" + "=" * 60)
if all_passed:
    print("✅ ALL VALIDATION CHECKS PASSED")
    print("   Dataset is ready for training!")
elif critical_passed:
    print("⚠️  VALIDATION PASSED WITH WARNINGS")
    print("   Dataset is usable but may benefit from improvements")
else:
    print("❌ VALIDATION FAILED")
    print("   Critical issues detected - review before training")
print("=" * 60)

In [ ]:
print("\n" + "=" * 60)
print("GENERATING DIAGNOSTIC VISUALIZATIONS")
print("=" * 60)

# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle(f'Loop Closure Dataset Diagnostics - Session {session_id}', fontsize=16, fontweight='bold')

# 1. Spatial distribution of pairs
ax = axes[0, 0]
for pair_type, color, label in [
    ('positive', 'green', 'Positive'),
    ('easy_negative', 'blue', 'Easy Negative'),
    ('hard_negative_type_a', 'orange', 'Hard Neg (Type A)'),
    ('hard_negative_type_b', 'red', 'Hard Neg (Type B)')
]:
    pairs_of_type = [d for d in dataset if d['pair_type'] == pair_type]
    if pairs_of_type:
        distances = [d['spatial_distance'] for d in pairs_of_type if d['spatial_distance'] is not None]
        if distances:
            ax.hist(distances, bins=20, alpha=0.6, color=color, label=label)

ax.set_xlabel('Spatial Distance (m)')
ax.set_ylabel('Count')
ax.set_title('Spatial Distance Distribution by Pair Type')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Feature value distribution
ax = axes[0, 1]
X_pos = X[y == 1]
X_neg = X[y == 0]
ax.hist(X_pos.flatten(), bins=50, alpha=0.6, color='green', label='Positive', density=True)
ax.hist(X_neg.flatten(), bins=50, alpha=0.6, color='red', label='Negative', density=True)
ax.set_xlabel('Pairwise Feature Value')
ax.set_ylabel('Density')
ax.set_title('Pairwise Feature Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Label balance
ax = axes[0, 2]
splits = ['Train', 'Val', 'Test', 'Overall']
pos_counts = [
    sum(d['label'] for d in train_dataset),
    sum(d['label'] for d in val_dataset),
    sum(d['label'] for d in test_dataset),
    int(np.sum(y))
]
neg_counts = [
    len(train_dataset) - pos_counts[0],
    len(val_dataset) - pos_counts[1],
    len(test_dataset) - pos_counts[2],
    len(dataset) - pos_counts[3]
]

x_pos = np.arange(len(splits))
width = 0.35
ax.bar(x_pos - width/2, pos_counts, width, label='Positive', color='green', alpha=0.7)
ax.bar(x_pos + width/2, neg_counts, width, label='Negative', color='red', alpha=0.7)
ax.set_xlabel('Split')
ax.set_ylabel('Count')
ax.set_title('Label Balance per Split')
ax.set_xticks(x_pos)
ax.set_xticklabels(splits)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 4. Map visualization (2D trajectory)
ax = axes[1, 0]
# Plot all nodes
node_positions = np.array([[n['pose']['x'], n['pose']['y']] for n in valid_nodes.values()])
ax.scatter(node_positions[:, 0], node_positions[:, 1], c='gray', s=10, alpha=0.5, label='Nodes')

# Plot positive pairs
for pair in positive_pairs[:50]:  # Limit to 50 for clarity
    node_i = valid_nodes[pair['query_node_id']]
    node_j = valid_nodes[pair['candidate_node_id']]
    ax.plot([node_i['pose']['x'], node_j['pose']['x']],
           [node_i['pose']['y'], node_j['pose']['y']],
           'g-', alpha=0.3, linewidth=1)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Map with Loop Closures (Green Lines)')
ax.axis('equal')
ax.grid(True, alpha=0.3)

# 5. Temporal coverage
ax = axes[1, 1]
train_times = [d['query_timestamp'] for d in train_dataset] + [d['candidate_timestamp'] for d in train_dataset]
val_times = [d['query_timestamp'] for d in val_dataset] + [d['candidate_timestamp'] for d in val_dataset]
test_times = [d['query_timestamp'] for d in test_dataset] + [d['candidate_timestamp'] for d in test_dataset]

ax.hist([train_times, val_times, test_times], bins=30, label=['Train', 'Val', 'Test'],
       color=['blue', 'orange', 'green'], alpha=0.6)
ax.axvline(train_end, color='red', linestyle='--', linewidth=2, label='Train/Val boundary')
ax.axvline(val_end, color='red', linestyle='--', linewidth=2, label='Val/Test boundary')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Count')
ax.set_title('Temporal Coverage of Splits')
ax.legend()
ax.grid(True, alpha=0.3)

# 6. Pair type distribution
ax = axes[1, 2]
pair_types = [d['pair_type'] for d in dataset]
type_counts = pd.Series(pair_types).value_counts()
colors = ['green', 'blue', 'orange', 'red']
ax.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
ax.set_title('Dataset Composition by Pair Type')

plt.tight_layout()
plt.savefig('dataset_diagnostics.png', dpi=150, bbox_inches='tight')
print("\n✅ Diagnostic plots saved to: dataset_diagnostics.png")
plt.show()

## <font color='#2E86AB'>▼ 14. Save Final Dataset</font>

Package all data into a single file ready for Phase 2 (Fusion MLP Training).

In [ ]:
print("=" * 60)
print("SAVING FINAL DATASET")
print("=" * 60)

# Create final dataset structure
final_dataset = {
    'metadata': {
        'session_id': session_id,
        'creation_date': datetime.now().isoformat(),
        'num_valid_nodes': len(valid_nodes),
        'num_total_pairs': len(dataset),
        'num_positive_pairs': len(positive_pairs),
        'num_easy_negative_pairs': len(easy_negative_pairs),
        'num_hard_negative_pairs': len(hard_negative_pairs),
        'positive_ratio': positive_ratio,
        'feature_dim': 1536,
        'combined_feature_dim': 1536,
        'camera_feature_dim': 1280,
        'lidar_feature_dim': 256,
        'duration_seconds': duration_sec,
        'map_dimensions': {'width': map_width, 'height': map_height}
    },
    'train': {
        'features': np.array([d['pairwise_feature'] for d in train_dataset]),
        'labels': np.array([d['label'] for d in train_dataset]),
        'pair_info': [{k: v for k, v in d.items() if k != 'pairwise_feature'} for d in train_dataset]
    },
    'val': {
        'features': np.array([d['pairwise_feature'] for d in val_dataset]),
        'labels': np.array([d['label'] for d in val_dataset]),
        'pair_info': [{k: v for k, v in d.items() if k != 'pairwise_feature'} for d in val_dataset]
    },
    'test': {
        'features': np.array([d['pairwise_feature'] for d in test_dataset]),
        'labels': np.array([d['label'] for d in test_dataset]),
        'pair_info': [{k: v for k, v in d.items() if k != 'pairwise_feature'} for d in test_dataset]
    },
    'node_database': valid_nodes,  # Keep for analysis/debugging
    'validation_report': {
        'checks': validation_checks,
        'all_passed': all_passed,
        'critical_passed': critical_passed
    }
}

# Save to pickle
output_filename = 'loop_closure_dataset.pkl'
with open(output_filename, 'wb') as f:
    pickle.dump(final_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"\n✅ Dataset saved to: {output_filename}")

# Get file size
file_size_mb = os.path.getsize(output_filename) / (1024 * 1024)
print(f"   File size: {file_size_mb:.2f} MB")

In [ ]:
print("\n" + "=" * 70)
print("FINAL DATASET GENERATION REPORT")
print("=" * 70)

report = f"""
📊 LOOP CLOSURE DATASET GENERATION - FINAL REPORT
{'='*70}

SESSION INFORMATION:
  • Session ID: {session_id}
  • Date generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
  • Duration: {duration_sec:.2f} seconds ({duration_sec/60:.2f} minutes)
  • Map dimensions: {map_width:.2f}m × {map_height:.2f}m

TRAJECTORY & FEATURES:
  • Total trajectory nodes: {len(trajectory_nodes)}
  • Valid nodes (both modalities): {len(valid_nodes)}
  • Camera alignment rate: {camera_alignment_rate:.1%}
  • LiDAR alignment rate: {lidar_alignment_rate:.1%}

CARTOGRAPHER CONSTRAINTS:
  • INTER_SUBMAP (loop closures): {len(inter_submap_constraints)}
  • Constraint-to-node match rate: {match_rate:.1%}

DATASET COMPOSITION:
  • Total pairs: {len(dataset)}
  • Positive pairs: {len(positive_pairs)} ({100*len(positive_pairs)/len(dataset):.1f}%)
  • Easy negative pairs: {len(easy_negative_pairs)} ({100*len(easy_negative_pairs)/len(dataset):.1f}%)
  • Hard negative pairs: {len(hard_negative_pairs)} ({100*len(hard_negative_pairs)/len(dataset):.1f}%)
      → Type A (perceptual): {len(hard_negative_pairs_type_a)}
      → Type B (viewpoint): {len(hard_negative_pairs_type_b)}

TRAIN/VAL/TEST SPLITS:
  • Train: {len(train_dataset)} pairs ({100*len(train_dataset)/len(dataset):.1f}%)
      → Positive: {sum(d['label'] for d in train_dataset)} ({100*train_pos_ratio:.1f}%)
  • Validation: {len(val_dataset)} pairs ({100*len(val_dataset)/len(dataset):.1f}%)
      → Positive: {sum(d['label'] for d in val_dataset)} ({100*val_pos_ratio:.1f}%)
  • Test: {len(test_dataset)} pairs ({100*len(test_dataset)/len(dataset):.1f}%)
      → Positive: {sum(d['label'] for d in test_dataset)} ({100*test_pos_ratio:.1f}%)

FEATURE STATISTICS:
  • Pairwise feature dimension: {X.shape[1]}D
  • Mean: {np.mean(X):.4f}
  • Std: {np.std(X):.4f}
  • Range: [{np.min(X):.4f}, {np.max(X):.4f}]

VALIDATION STATUS:
  {'✅' if all_passed else '⚠️ ' if critical_passed else '❌'} Overall: {'PASSED' if all_passed else 'PASSED WITH WARNINGS' if critical_passed else 'FAILED'}
"""

for check_name, check_result in validation_checks:
    report += f"  {'✅' if check_result else '❌'} {check_name}\n"

report += f"""
OUTPUT FILES:
  • Dataset: {output_filename} ({file_size_mb:.2f} MB)
  • Diagnostics: dataset_diagnostics.png
  • Cartographer validation: cartographer_validation_report.txt

NEXT STEPS:
  1. Load dataset with: pickle.load(open('{output_filename}', 'rb'))
  2. Train Fusion MLP (Phase 2): 1536→512→128→1 architecture
  3. Use BCE loss + hard negative mining
  4. Monitor validation performance
  5. Export to ONNX/TensorRT for Jetson Nano deployment

{'='*70}
"""

print(report)

# Save report
with open('dataset_generation_report.txt', 'w') as f:
    f.write(report)

print("\n✅ Final report saved to: dataset_generation_report.txt")

print("\n" + "=" * 70)
if all_passed:
    print("🎉 DATASET GENERATION COMPLETE - READY FOR TRAINING!")
elif critical_passed:
    print("✅ DATASET GENERATION COMPLETE - USABLE WITH WARNINGS")
else:
    print("⚠️  DATASET GENERATION COMPLETE - REVIEW VALIDATION ISSUES")
print("=" * 70)